In [1]:
import numpy as np
import pandas as pd
import plotly

#Data Source
import yfinance as yf

#Data viz
import plotly.graph_objs as go
import sqlite3

conn = sqlite3.connect("stock.sqlite")
cur = conn.cursor()
ticker1 = 'AOT.BK'
ticker2 = 'ACE.BK'
ticker3 = 'KEX.BK'
ticker4 = 'EPG.BK'
ticker5 = 'AAV.BK'
ticker6 = 'DELTA.BK'
query1 = "select * from stock_table_hr where `ticker` == '%s'" % ticker1
query2 = "select * from stock_table_d where `ticker` == '%s' and datetime > '2023-01-04' and datetime < '2023-01-06'" % ticker2
query3 = "select * from stock_table_d where `ticker` == '%s' and datetime > '2023-01-05' and datetime < '2023-01-06'" % ticker3
query4 = "select * from stock_table_d where `ticker` == '%s' and datetime > '2023-01-05' and datetime < '2023-01-06'" % ticker4
query5 = "select * from stock_table_d where `ticker` == '%s' and datetime > '2023-01-05' and datetime < '2023-01-06'" % ticker5
query6 = "select * from stock_table_d where `ticker` == '%s' and datetime > '2022-12-20' and datetime < '2023-01-06'" % ticker6
AOT = pd.read_sql(query1,conn)
ACE = pd.read_sql(query2,conn)
KEX = pd.read_sql(query3,conn)
EPG = pd.read_sql(query4,conn)
AAV = pd.read_sql(query5,conn)
DELTA = pd.read_sql(query6,conn)
x = [ticker1,ticker2,ticker3,ticker4,ticker5,ticker6]
count = 0
#declare figure
fig = go.Figure()
DELTA['time'] = pd.to_datetime(DELTA['Datetime'])
DELTA.set_index(DELTA['time'],inplace = True)

#Candlestick
for column in [AOT,ACE,KEX,EPG,AAV,DELTA]:
    # name = column.Ticker[0]
    fig.add_trace(go.Candlestick(x=column.Datetime,
                    open=column.Open,
                    high=column.High,
                    low=column.Low,
                    close=column.Close, 
                    name = x[count],))
    count += 1
    
df_resample = DELTA.resample('H').max()
merged_index  = DELTA.index.append(df_resample.index)
timegap = merged_index[~merged_index.duplicated(keep = False)]
fig.update_xaxes( rangebreaks=[ dict(values = timegap, dvalue = 3600000) ])

# Add titles
fig.update_layout(
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=list(
            [dict(label = 'Disable',
                method = 'update',
                args = [{'visible': [False, False, False, False, False]}, # the index of True aligns with the indices of plot traces
                        {'title': 'Disable',
                        'showlegend':True}]),

            dict(label = 'AOT',
                method = 'update',
                args = [{'visible': [True, False, False, False, False, False]}, # the index of True aligns with the indices of plot traces
                        {'title': 'AOT',
                        'showlegend':True}]),
            dict(label = 'ACE',
                method = 'update',
                args = [{'visible': [False, True, False, False, False, False]},
                        {'title': 'ACE',
                        'showlegend':True}]),
            dict(label = 'KEX',
                method = 'update',
                args = [{'visible': [False, False, True, False, False, False]},
                        {'title': 'KEX',
                        'showlegend':True}]),
            dict(label = 'EPG',
                method = 'update',
                args = [{'visible': [False, False, False, True, False, False]},
                        {'title': 'EPG',
                        'showlegend':True}]),
            dict(label = 'AAV',
                  method = 'update',
                  args = [{'visible': [False, False, False, False, True, False]},
                          {'title': 'AAV',
                           'showlegend':True}]),
            dict(label = 'DELTA',
                  method = 'update',
                  args = [{'visible': [False, False, False, False, False, True]},
                          {'title': 'DELTA',
                           'showlegend':True}]),
            ])
        )
    ])

fig.update_traces(visible="legendonly")
fig.update_yaxes(fixedrange=False)
# X-Axes
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=15, label="15m", step="minute", stepmode="backward"),
            dict(count=45, label="45m", step="minute", stepmode="backward"),
            dict(count=1, label="HTD", step="hour", stepmode="todate"),
            dict(count=3, label="3h", step="hour", stepmode="backward"),
            dict(step="all")
        ])
    )
)

#Show
fig.show()

In [10]:
DELTA['time'] = pd.to_datetime(DELTA['Datetime'])
DELTA.set_index(DELTA['time'],inplace = True)
df_resample = DELTA.resample('H').max()
merged_index  = DELTA.index.append(df_resample.index)
timegap = merged_index[~merged_index.duplicated(keep = False)]
print(timegap)

DatetimeIndex(['2022-12-01 01:00:00', '2022-12-01 02:00:00',
               '2022-12-01 03:00:00', '2022-12-01 04:00:00',
               '2022-12-01 05:00:00', '2022-12-01 06:00:00',
               '2022-12-01 07:00:00', '2022-12-01 08:00:00',
               '2022-12-01 09:00:00', '2022-12-01 10:00:00',
               ...
               '2023-01-04 14:00:00', '2023-01-04 15:00:00',
               '2023-01-04 16:00:00', '2023-01-04 17:00:00',
               '2023-01-04 18:00:00', '2023-01-04 19:00:00',
               '2023-01-04 20:00:00', '2023-01-04 21:00:00',
               '2023-01-04 22:00:00', '2023-01-04 23:00:00'],
              dtype='datetime64[ns]', name='time', length=818, freq=None)


In [12]:
from datetime import timedelta
import yfinance as yf
import plotly.graph_objects as go

df = yf.download(tickers='DELTA.BK', period='1mo', interval='1d')
fig = go.Figure()
fig.add_trace(go.Candlestick(x=df.index,
                open=df.Open,
                high=df.High,
                low=df.Low,
                close=df.Close, name = 'market data'))

df_resample = df.resample('H').max()
merged_index  = df.index.append(df_resample.index)
timegap = merged_index[~merged_index.duplicated(keep = False)]

fig.update_xaxes( rangebreaks=[ dict(values = timegap, dvalue = 3600000) ])

# alldays =set(data.Datetime[0]+timedelta(x) for x in range((data.Datetime[len(data.Datetime)-1]- data.Datetime[0]).days))
# missing=sorted(set(alldays)-set(data.Datetime))
# fig.update_xaxes(rangebreaks=[dict(values=missing)])

# fig.update_xaxes(
#         rangeslider_visible=True,
#         rangebreaks=[
#             # NOTE: Below values are bound (not single values), ie. hide x to y
#             dict(bounds=["sat", "mon"]),  # hide weekends, eg. hide sat to before mon
#             # dict(bounds=[16.5, 8.5], pattern="hour"),  # hide hours outside of 8.30am-4.30pm
#         ]
#     )

fig.update_yaxes(fixedrange=False)
fig.show()
print(timegap)

[*********************100%***********************]  1 of 1 completed


DatetimeIndex(['2022-12-13 01:00:00', '2022-12-13 02:00:00',
               '2022-12-13 03:00:00', '2022-12-13 04:00:00',
               '2022-12-13 05:00:00', '2022-12-13 06:00:00',
               '2022-12-13 07:00:00', '2022-12-13 08:00:00',
               '2022-12-13 09:00:00', '2022-12-13 10:00:00',
               ...
               '2023-01-10 14:00:00', '2023-01-10 15:00:00',
               '2023-01-10 16:00:00', '2023-01-10 17:00:00',
               '2023-01-10 18:00:00', '2023-01-10 19:00:00',
               '2023-01-10 20:00:00', '2023-01-10 21:00:00',
               '2023-01-10 22:00:00', '2023-01-10 23:00:00'],
              dtype='datetime64[ns]', name='Date', length=676, freq=None)
